In [1]:
import pandas as pd

In [3]:
#Load the Wikipedia table
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
tables = pd.read_html(url)

In [4]:
tables[0]

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...
498,XYL,Xylem Inc.,Industrials,Industrial Machinery & Supplies & Components,"White Plains, New York",2011-11-01,1524472,2011
499,YUM,Yum! Brands,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
500,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
501,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927


In [5]:
#Find the table that contains the 'Date added' column
sp500_additions = None
for table in tables:
    if 'Date added' in table.columns:
        sp500_additions = table
        break

#Drop rows with missing 'Date added'
sp500_additions = sp500_additions.dropna(subset=["Date added"])

#Convert 'Date added' to datetime and extract the year
sp500_additions["Year added"] = pd.to_datetime(sp500_additions["Date added"]).dt.year
sp500_additions.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded,Year added
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902,1957
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916,2017
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888,1957
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888),2012
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989,2011


In [11]:
#Exclude the year 1957
sp500_additions = sp500_additions[sp500_additions["Year added"] != 1957]

#Count the number of additions per year
additions_per_year = sp500_additions["Year added"].value_counts().sort_index()
# additions_per_year

In [13]:
sp500_additions["Year added"].value_counts().head()

,count
Year added,
2017,23
2016,23
2019,22
2008,17
2024,16


In [10]:
#Identify the year with the most additions
most_additions_year = additions_per_year.idxmax()
most_additions_count = additions_per_year.max()

print(f"Year with the most additions: {most_additions_year} ({most_additions_count} companies)")

Year with the most additions: 2016 (23 companies)


In [2]:
!pip -q install yfinance

In [8]:
import yfinance as yf
from datetime import datetime

In [13]:
from datetime import datetime

END = pd.Timestamp("2025-05-01")
START_YTD = pd.Timestamp(f"{END.year}-01-01")

TICKERS = {
    "United States - S&P 500": "^GSPC",
    "China - Shanghai Composite": "000001.SS",
    "Hong Kong - Hang Seng": "^HSI",
    "Australia - ASX 200": "^AXJO",
    "India - Nifty 50": "^NSEI",
    "Canada - TSX Composite": "^GSPTSE",
    "Germany - DAX": "^GDAXI",
    "United Kingdom - FTSE 100": "^FTSE",
    "Japan - Nikkei 225": "^N225",
    "Mexico - IPC Mexico": "^MXX",
    "Brazil - Ibovespa": "^BVSP"
}

HORIZONS = {
    "YTD":  (START_YTD, END),
    "3Y":   (END - pd.DateOffset(years=3),  END),
    "5Y":   (END - pd.DateOffset(years=5),  END),
    "10Y":  (END - pd.DateOffset(years=10), END),
}

EARLIEST = min(start for start, _ in HORIZONS.values())
print(f"Earliest date to download: {EARLIEST:%Y-%m-%d}")


Earliest date to download: 2015-05-01


In [19]:
# ---- Bulk download -------------------------------------------------
raw = yf.download(
    list(TICKERS.values()),
    start=EARLIEST,
    end=END,
    progress=False,
    group_by="ticker",      # 1st level = ticker
    auto_adjust=False       # keep both 'Close' and 'Adj Close' if available
)

# ---- Choose price field -------------------------------------------
lvl1 = raw.columns.get_level_values(1).unique()
pick = "Adj Close" if "Adj Close" in lvl1 else "Close"
print(f"Using price field: {pick}")

prices = (
    raw.xs(pick, axis=1, level=1)
       .rename(columns={v: k for k, v in TICKERS.items()})
       .dropna(how="all")
)

print("Price table shape:", prices.shape)



Using price field: Adj Close
Price table shape: (2606, 11)


In [20]:
prices.tail()

Ticker,Germany - DAX,India - Nifty 50,Australia - ASX 200,United States - S&P 500,Canada - TSX Composite,Hong Kong - Hang Seng,Japan - Nikkei 225,Mexico - IPC Mexico,China - Shanghai Composite,Brazil - Ibovespa,United Kingdom - FTSE 100
Date,,,,,,,,,,,
2025-04-24,+2206450.98%,+2424669.92%,+796820.02%,+548477.00%,+2472750.00%,+2190975.98%,+3503914.84%,+5638200.00%,+329728.81%,+13458000.00%,+840740.04%
2025-04-25,+2224244.92%,+2403934.96%,NaN,+552521.00%,+2471050.00%,+2198074.02%,+3570573.83%,+5672012.11%,+329506.01%,+13473900.00%,+841529.98%
2025-04-28,+2227166.99%,+2432850.00%,+799710.01%,+552875.00%,+2479859.96%,+2197196.09%,+3583998.83%,+5698012.89%,+328841.50%,+13501600.00%,+841729.98%
2025-04-29,+2242583.01%,+2433594.92%,+807060.01%,+556083.01%,+2487450.00%,+2200810.94%,NaN,+5561342.97%,+328665.50%,+13509300.00%,+846350.00%
2025-04-30,+2249698.05%,+2433419.92%,+812620.02%,+556906.01%,+2484169.92%,+2211941.02%,+3604537.89%,+5625928.12%,+327903.10%,+13506700.00%,+849490.04%


In [21]:
# ---------- helper functions (revised) -----------------
def clean_prices(df: pd.DataFrame, end: pd.Timestamp) -> pd.DataFrame:
    """
    Forward-fill to `end`, back-fill just once (in case start date is a holiday),
    and keep only rows inside [earliest, end].
    """
    out = df.ffill(limit=5).bfill(limit=1)          # small gaps only
    # make sure we have a row exactly on `end`
    if out.index.max() < end:
        last_row = out.iloc[[-1]].copy()
        last_row.index = [end]
        out = pd.concat([out, last_row])
    return out

def period_return(series: pd.Series,
                  start: pd.Timestamp,
                  end: pd.Timestamp) -> float:
    seg = series.loc[start:end].dropna()
    if seg.empty:
        return float("nan")
    return (seg.iloc[-1] / seg.iloc[0]) - 1


In [22]:
# def period_return(series: pd.Series, start: pd.Timestamp, end: pd.Timestamp) -> float:
#     start_px = series.loc[series.index >= start].iloc[0]
#     end_px   = series.loc[series.index <= end].iloc[-1]
#     return (end_px / start_px) - 1

def horizon_returns(df: pd.DataFrame, horizons: dict) -> pd.DataFrame:
    out = {}
    for label, (start, end) in horizons.items():
        out[label] = df.apply(period_return, axis=0, start=start, end=end)
    return pd.DataFrame(out)


In [23]:
prices = clean_prices(prices, END)

In [24]:
returns = horizon_returns(prices, HORIZONS).T.round(4)

benchmark = returns["United States - S&P 500"]
beats = (returns.drop(columns="United States - S&P 500")
                 .gt(benchmark, axis=0)
                 .sum(axis=1))

pd.options.display.float_format = "{:+.2%}".format

print("Indexes that beat the S&P 500")
print(beats.to_frame("Count > S&P"))

print("\nTotal returns by horizon")
returns


Indexes that beat the S&P 500
     Count > S&P
YTD            9
3Y             3
5Y             2
10Y            1

Total returns by horizon


Ticker,Germany - DAX,India - Nifty 50,Australia - ASX 200,United States - S&P 500,Canada - TSX Composite,Hong Kong - Hang Seng,Japan - Nikkei 225,Mexico - IPC Mexico,China - Shanghai Composite,Brazil - Ibovespa,United Kingdom - FTSE 100
YTD,+13.00%,+2.49%,-0.40%,-5.31%,+0.46%,+10.27%,-9.65%,+13.62%,-2.17%,+12.29%,+3.94%
3Y,+61.40%,+42.56%,+10.61%,+34.02%,+20.05%,+4.88%,+34.40%,+8.43%,+7.61%,+26.66%,+12.60%
5Y,+107.12%,+146.80%,+54.91%,+96.74%,+69.91%,-10.24%,+83.72%,+54.26%,+14.65%,+67.77%,+47.40%
10Y,+93.61%,+192.06%,+39.76%,+164.15%,+61.94%,-21.35%,+84.55%,+24.36%,-26.81%,+135.50%,+21.60%
